In [1]:
with open("Day10.txt") as file:
    data = file.read()

machines = []
for line in data.splitlines():
    ls, *bs, js = line.split()
    machines.append({
        "lights": {i for i, l in enumerate(ls[1:-1]) if l == "#"},
        "buttons": tuple(set(eval(b[1:-1] + ",")) for b in bs),
        "joltages": eval(js[1:-1] + ","),
    })

In [2]:
from collections import deque

def solve1(lights, buttons, joltages=None):
    todo, done = deque(), set()
    todo.append((set(), 0))
    while todo:
        state, steps = todo.popleft()
        if state == lights:
            return steps
        for toggles in buttons:
            newstate = state ^ toggles
            if newstate in done:
                continue
            done.add(frozenset(newstate))
            todo.append((newstate, steps + 1))

sum(solve1(**machine) for machine in machines)

522

In [3]:
import z3

def solve2(joltages, buttons, lights=None):
    toggles = [z3.Int(f"b{i}") for i in range(len(buttons))]
    solver = z3.Optimize()
    for toggle in toggles:
        solver.add(toggle >= 0)
    for i, joltage in enumerate(joltages):
        solver.add(sum(toggles[j] for j, button in enumerate(buttons) if i in button) == joltage)
    solver.minimize(sum(toggles))
    if solver.check() == z3.sat:
        model = solver.model()
        return sum(model[toggle].as_long() for toggle in toggles)

sum(solve2(**machine) for machine in machines)

18105